### **Curating Heart-AVN-atrial-cardiomyocyte.h5ad**

Article: Spatially resolved multiomics of human cardiac niches

DOI: https://doi.org/10.1038/s41586-023-06311-1 

Data Source : https://www.heartcellatlas.org

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math
import re

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/heart_cell_atlas/data/Heart-AVN-atrial-cardiomyocyte.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 955 × 32732
    obs: 'sangerID', 'donor', 'donor_type', 'region', 'age', 'gender', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'flushed', 'cell_state', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'scrublet_score'
    var: 'gene_name_scRNA-0-original', 'gene_name_snRNA-1-original', 'gene_name_multiome-2-original', 'gene_id'
    uns: 'cell_state_colors'
    obsm: 'X_umap'

##### **X - expression matrix**

In [7]:
# View the expression matrix of the anndata object.

In [8]:
adata.X

<955x32732 sparse matrix of type '<class 'numpy.float32'>'
	with 3297954 stored elements in Compressed Sparse Row format>

In [9]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [10]:
print(adata.X)

  (0, 32696)	1.3670148
  (0, 32695)	0.6803355
  (0, 32692)	1.1608717
  (0, 32691)	0.5486444
  (0, 32690)	0.90089697
  (0, 14132)	0.99526656
  (0, 14080)	1.3029088
  (0, 13948)	0.6803355
  (0, 13854)	1.4272577
  (0, 13850)	0.79668486
  (0, 13791)	1.9238458
  (0, 13767)	1.2344099
  (0, 13760)	0.6803355
  (0, 13655)	0.79668486
  (0, 13614)	2.1173456
  (0, 13598)	2.0252686
  (0, 13599)	2.7791529
  (0, 13597)	2.837903
  (0, 13567)	1.6374023
  (0, 13566)	1.4840766
  (0, 13536)	1.9588089
  (0, 13447)	1.4272577
  (0, 13327)	2.0875854
  (0, 13312)	0.79668486
  (0, 13308)	0.6803355
  :	:
  (954, 7247)	4.7800493
  (954, 7233)	4.933
  (954, 7215)	3.0294108
  (954, 7187)	3.0294108
  (954, 7182)	3.0294108
  (954, 7116)	3.0294108
  (954, 7112)	3.0294108
  (954, 7083)	3.0294108
  (954, 7081)	3.698089
  (954, 7048)	3.0294108
  (954, 7046)	3.0294108
  (954, 7034)	3.0294108
  (954, 7029)	4.378773
  (954, 6723)	3.0294108
  (954, 6596)	3.0294108
  (954, 6392)	3.0294108
  (954, 6113)	3.0294108
  (954, 6103)

##### **Raw counts matrix**

In [11]:
# If X has normalized counts, check for the raw counts matrix.

In [12]:
# check whether raw counts are present in adata.raw

In [13]:
adata.raw

In [14]:
# print(adata.raw.X)

In [15]:
#Raw count matrix is not present in adata.raw. Here the raw counts are provided in a separate object, load the raw counts matrix

In [ ]:
araw = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/heart_cell_atlas/Raw_counts/RNA_adult-8reg_full_raw_cellstate-annotated.h5ad')

In [ ]:
araw

In [ ]:
# get raw count matrix only for this data

In [ ]:
araw = araw[araw.obs['region']=='AVN']

In [ ]:
araw = araw[araw.obs['cell_type']=='Atrial Cardiomyocyte']

In [ ]:
# Check whether adata and araw has same dimensions.

In [ ]:
araw.X

In [ ]:
# print raw matrix

In [ ]:
print(araw.X)

##### **Variables(var)**

In [ ]:
# View the var of anndata and raw object

In [ ]:
adata.var

In [ ]:
araw.var

In [ ]:
# If ensembl ids are not in the index column and is present in another column, set the ensembl ids column as the index column.

In [ ]:
adata.var['gene_symbols'] = adata.var_names

In [ ]:
adata.var

In [ ]:
adata.var_names = adata.var['gene_id']

In [ ]:
adata.var

In [ ]:
# Load the approved genes file.

In [ ]:
approved_genes = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/gene_info/genes_approved.csv')

In [ ]:
#Create a dictionary from the approved genes file 

In [ ]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [ ]:
genedict

In [ ]:
len(genedict)

In [ ]:
#Filter out the genes which are not in the approved genes file.

In [ ]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [ ]:
len(var_to_keep_adata)

In [ ]:
len(var_to_keep_araw)

In [ ]:
adata.var

In [ ]:
araw.var

In [ ]:
# Modify the anndata object by filtering out the filtered genes.

In [ ]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [ ]:
adata.var

In [ ]:
araw.var

feature is filtered

In [ ]:
#Since feature was not filtered out in the normalized matrix (X), assign false

In [ ]:
adata.var['feature_is_filtered'] = [False] * len(adata.var)

In [ ]:
#View var

In [ ]:
adata.var

In [ ]:
araw.var

In [ ]:
del araw.var['gene_name-new']
del araw.var['gene_name_scRNA-0-original']
del araw.var['gene_name_snRNA-1-original']
del araw.var['gene_name_multiome-2-original']

In [ ]:
del adata.var['gene_name_scRNA-0-original']
del adata.var['gene_name_snRNA-1-original']
del adata.var['gene_name_multiome-2-original']
del adata.var['gene_id']
del adata.var['gene_symbols']

In [ ]:
# View var

In [ ]:
adata.var

In [ ]:
araw.var

#### **obs (Cell metadata)**

In [ ]:
#view obs

In [ ]:
adata.obs

In [ ]:
# view the column names in obs

In [ ]:
adata.obs.columns

#### **assay_ontology_term_id**

In [ ]:
# identify the column in adata which corresponds to assay

In [ ]:
list(adata.obs['kit_10x'].unique())

In [ ]:
# create a dictionary with assay and corresponding assay ontology term id

In [ ]:
mapping= {'3prime-v2':'EFO:0009899', '3prime-v3':'EFO:0009922', 'Multiome-v1' : 'EFO:0030059'}

In [ ]:
# add the assay_ontology_term_id column

In [ ]:
adata.obs['assay_ontology_term_id'] = adata.obs['kit_10x'].map(mapping)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [ ]:
# view adata.obs

In [ ]:
adata.obs

#### **cell_type_ontology_term_id**

In [ ]:
#identify the column in adata.obs related. to cell type annotation

In [ ]:
adata.obs.columns

In [ ]:
list(adata.obs['cell_state'].unique())

In [ ]:
# add the cell_type_ontology_term_id column

In [ ]:
adata.obs['cell_type_ontology_term_id'] = ['CL:0002129']* len(adata.obs)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [ ]:
# view adata.obs

In [ ]:
adata.obs

#### **development_stage_ontology_term_id**

In [ ]:
# identify the column in adata which corresponds to age

In [ ]:
adata.obs.columns

In [ ]:
list(adata.obs['age'].unique())

In [ ]:
# modify the age of donors based on supplementary info

In [ ]:
adata.obs['age'] = np.where(adata.obs['donor'] == 'D6' , '70-75', adata.obs['age'])
adata.obs['age'] = np.where(adata.obs['donor'] == 'AH1' , '45-50', adata.obs['age'])
adata.obs['age'] = np.where(adata.obs['donor'] == 'A61' , '70-75', adata.obs['age'])

In [ ]:
# view the modified age

In [ ]:
age_value = adata.obs.loc[adata.obs['donor'] == 'A61', 'age'].values[0]

In [ ]:
age_value

In [ ]:
# Get unique values of age column

In [ ]:
list(adata.obs['age'].unique())

In [ ]:
# create a dictionary for age and development stage ontology term id

In [ ]:
mapping= {'50-55':'HsapDv:0000240', 
          '55-60':'HsapDv:0000240', 
          '70-75':'HsapDv:0000242', 
          '65-70':'HsapDv:0000241', 
          '60-65':'HsapDv:0000241',
          '40-45':'HsapDv:0000239', 
          '45-50':'HsapDv:0000239', 
          '20-25':'HsapDv:0000237'}

In [ ]:
# add the development_stage_ontology_term_id column

In [ ]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['age'].map(mapping)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [ ]:
# view unique values of development_stage_ontology_term_id column

In [ ]:
list(adata.obs['development_stage_ontology_term_id'].unique())

In [ ]:
# view adata.obs

In [ ]:
adata.obs

#### **donor_id**

In [ ]:
#identify the column in adata.obs which provides donor information

In [ ]:
adata.obs.columns

In [ ]:
list(adata.obs['donor'].unique())

In [ ]:
# add the donor_id column

In [ ]:
adata.obs['donor_id'] = adata.obs['donor']

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [ ]:
# view unique values of donor_id column

In [ ]:
list(adata.obs['donor_id'].unique())

In [ ]:
#view obs

In [ ]:
adata.obs

#### **disease_ontology_term_id**

In [ ]:
# Assign normal since all are healthy patients

In [ ]:
# add the disease_ontology_term_id column

In [ ]:
adata.obs['disease_ontology_term_id'] = ['PATO:0000461']* len(adata.obs)

In [ ]:
# change datatype of the column

In [ ]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **is_primary_data**

In [ ]:
adata.obs['is_primary_data'] = [False]* len(adata.obs)

In [ ]:
# view 'is_primary_data' unique values

In [ ]:
list(adata.obs['is_primary_data'].unique())

In [ ]:
#change data type of column

In [ ]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **organism_ontology_term_id**

In [ ]:
# assign organism id 

In [ ]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [ ]:
#change data type of column

In [ ]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **self_reported_ethnicity_ontology_term_id**

In [ ]:
# read the supplementary information from article

In [ ]:
# create a dictionary of donor and ethinic_origin

In [ ]:
mapping= {'D1': 'Caucasian',
 'D2': 'Caucasian',
 'D3': 'Caucasian',
 'D4': 'Caucasian',
 'D5': 'Caucasian',
 'D6': 'Caucasian',
 'D7': 'Caucasian',
 'D11': 'Caucasian',
 'H2': 'Caucasian',
 'H3': 'Asian',
 'H4': 'Caucasian',
 'H5': 'Caucasian',
 'H6': 'Caucasian',
 'H7': 'Caucasian',
 'A61': 'Caucasian',
 'AH1': 'Caucasian',
 'AH1-A61': 'unknown',
 'AH2': 'South Asian',
 'AV10': 'Caucasian',
 'AV13': 'Caucasian',
 'AV14': 'Caucasian',
 'AV3': 'Caucasian',
 'D8': 'Caucasian',
 'AH5': 'Caucasian',
 'AH6': 'Caucasian',
 'AV1': 'Caucasian'}

In [ ]:
# add ethinic_origin column

In [ ]:
adata.obs['ethinic_origin'] = adata.obs['donor'].map(mapping)

In [ ]:
mapping= {'Caucasian': 'HANCESTRO:0005',
 'Asian': 'HANCESTRO:0008',
 'unknown': 'unknown',
 'South Asian': 'HANCESTRO:0006'}

In [ ]:
# add self_reported_ethnicity_ontology_term_id column

In [ ]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['ethinic_origin'].map(mapping)

In [ ]:
# change data type

In [ ]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **sex_ontology_term_id**

In [ ]:
# identify the column in adata.obs which corresponds to sex

In [ ]:
adata.obs.columns

In [ ]:
# list the unique values 

In [ ]:
list(adata.obs['gender'].unique())

In [ ]:
# create a dictionary of sex and sex ontology term id

In [ ]:
mapping= {'Female': 'PATO:0000383', 'Male': 'PATO:0000384'}

In [ ]:
# add sex_ontology_term_id column

In [ ]:
adata.obs['sex_ontology_term_id'] = adata.obs['gender'].map(mapping)

In [ ]:
# change data type

In [ ]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [ ]:
adata.obs

#### **suspension_type**

In [ ]:
# identify the column in adata.obs which corresponds to suspension type

In [ ]:
adata.obs.columns

In [ ]:
# list the unique values in the column

In [ ]:
list(adata.obs['cell_or_nuclei'].unique())

In [ ]:
# create a mapping dictionary

In [ ]:
mapping= {'Cell':'cell', 'Nuclei':'nucleus'}

In [ ]:
# add 'suspension_type' column

In [ ]:
adata.obs['suspension_type'] = adata.obs['cell_or_nuclei'].map(mapping)

In [ ]:
# change data type of column

In [ ]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [ ]:
# view obs

In [ ]:
adata.obs

#### **tissue_ontology_term_id**

In [ ]:
# identify the column in adata.obs which corresponds to tissue

In [ ]:
adata.obs.columns

In [ ]:
list(adata.obs['region'].unique())

In [ ]:
# create a dictionary with tissue and corresponding tissue ontology term id

In [ ]:
mapping= {'AX':'UBERON:0002098', 
          'LV':'UBERON:0002084', 
          'RV':'UBERON:0002080', 
          'LA':'UBERON:0002079', 
          'SP':'UBERON:0002094', 
          'RA':'UBERON:0002078', 
          'SAN':'UBERON:0002351', 
          'AVN':'UBERON:0002352'}

In [ ]:
# add 'tissue_ontology_term_id' column

In [ ]:
adata.obs['tissue_ontology_term_id'] = adata.obs['region'].map(mapping)

In [ ]:
# change data type of column

In [ ]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [ ]:
#list the unique values in 'tissue_ontology_term_id' column

In [ ]:
list(adata.obs['tissue_ontology_term_id'].unique())

In [ ]:
# view obs

In [ ]:
adata.obs

#### **obsm (Embeddings)**

In [ ]:
# view obsm

In [ ]:
# check whether all columns are prefixed with X

In [ ]:
adata.obsm

#### **uns (Dataset Metadata)**

In [ ]:
# View

In [ ]:
adata.uns

In [ ]:
adata.uns.keys

In [ ]:
# Give a title for the dataset

In [ ]:
adata.uns['title'] = 'snRNA-seq data - Heart AVN atrial cardiomyocytes'

In [ ]:
# Set the default embedding

In [ ]:
adata.uns['default_embedding'] = 'X_umap'

### **Final check**

In [ ]:
# view anndata object

In [ ]:
adata

In [ ]:
# view obs and var data types

In [ ]:
adata.obs.dtypes

In [ ]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [ ]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

In [ ]:
# view obs

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
# delete unwanted columns in obs

In [ ]:
del adata.obs['donor']
del adata.obs['cell_or_nuclei']
del adata.obs['kit_10x']
del adata.obs['gender']
del adata.obs['ethinic_origin']
del adata.obs['modality']

In [ ]:
# view obs

In [ ]:
adata.obs

In [ ]:
adata.obs.columns

In [ ]:
# view var

In [ ]:
adata.var

In [ ]:
#view uns

In [ ]:
adata.uns

In [ ]:
list(adata.uns.keys())

In [ ]:
adata.obs.columns

In [ ]:
#check the format of expression matrix

In [ ]:
adata.X

In [ ]:
araw.X

In [ ]:
#Copy raw counts to adata.raw

In [ ]:
adata.raw = araw

In [ ]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/heart_cell_atlas/final_objects/Heart-AVN-atrial-cardiomyocyte.h5ad', compression = 'gzip')